In [55]:
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import finnhub
import time
from datetime import datetime, timedelta
import re
from transformers import pipeline
import ta
from ta.momentum import RSIIndicator
from ta.trend import MACD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")



In [7]:
stock_df = yf.download("TSLA", start="2015-01-01", end = "2025-07-19")  #stock price data

/tmp/ipython-input-7-2883391446.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stock_df = yf.download("TSLA", start="2015-01-01", end = "2025-07-19")  #stock price data
[*********************100%***********************]  1 of 1 completed


In [8]:
stock_df.head()

Price,Close,High,Low,Open,Volume
Ticker,TSLA,TSLA,TSLA,TSLA,TSLA
Date,,,,,
2015-01-02,14.620667,14.883333,14.217333,14.858000,71466000
2015-01-05,14.006000,14.433333,13.810667,14.303333,80527500
2015-01-06,14.085333,14.280000,13.614000,14.004000,93928500
2015-01-07,14.063333,14.318667,13.985333,14.223333,44526000
2015-01-08,14.041333,14.253333,14.000667,14.187333,51637500


In [9]:
stock_df.isnull().sum()

,,0
Price,Ticker,
Close,TSLA,0
High,TSLA,0
Low,TSLA,0
Open,TSLA,0
Volume,TSLA,0


In [10]:
stock_df.shape

(2651, 5)

In [11]:
stock_df.columns = [col[0] if col[0] != 'Date' else 'Date' for col in stock_df.columns]
stock_df = stock_df.reset_index()


In [12]:
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2651 entries, 0 to 2650
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2651 non-null   datetime64[ns]
 1   Close   2651 non-null   float64       
 2   High    2651 non-null   float64       
 3   Low     2651 non-null   float64       
 4   Open    2651 non-null   float64       
 5   Volume  2651 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 124.4 KB


In [13]:
  stock_df['Date'] = pd.to_datetime(stock_df['Date']).dt.date


In [14]:
stock_df.head()

,Date,Close,High,Low,Open,Volume
0,2015-01-02,14.620667,14.883333,14.217333,14.858000,71466000
1,2015-01-05,14.006000,14.433333,13.810667,14.303333,80527500
2,2015-01-06,14.085333,14.280000,13.614000,14.004000,93928500
3,2015-01-07,14.063333,14.318667,13.985333,14.223333,44526000
4,2015-01-08,14.041333,14.253333,14.000667,14.187333,51637500


In [15]:

# Your Finnhub API key
api_key = 'd1tqrqpr01qt0evco7cgd1tqrqpr01qt0evco7d0'  # Replace with your real key
finnhub_client = finnhub.Client(api_key=api_key)

# Generate 3-month intervals from 2015 to 2025
def generate_date_ranges(start_date, end_date, interval_days=90):
    ranges = []
    current = start_date
    while current < end_date:
        next_date = min(current + timedelta(days=interval_days), end_date)
        ranges.append((current.strftime('%Y-%m-%d'), next_date.strftime('%Y-%m-%d')))
        current = next_date + timedelta(days=1)
    return ranges

# Setup
start_date = datetime(2015, 1, 1)
end_date = datetime(2025, 7, 19)  # today

date_ranges = generate_date_ranges(start_date, end_date)

# Pull news data for each range
all_news = []
for start, end in date_ranges:
    print(f"📅 Fetching news from {start} to {end}...")
    try:
        news = finnhub_client.company_news('TSLA', _from=start, to=end)
        all_news.extend(news)
    except Exception as e:
        print(f"❌ Error during {start} to {end}: {e}")
    # Wait if needed due to rate limit (free tier)
    time.sleep(1.1)

# Convert to DataFrame
news_df = pd.DataFrame(all_news).drop_duplicates(subset='id')

# Convert time
if 'datetime' in news_df.columns:
    news_df['datetime'] = pd.to_datetime(news_df['datetime'], unit='s')

print(f"\n✅ Total articles collected: {len(news_df)}")
print(news_df[['datetime', 'headline', 'summary']].head())


📅 Fetching news from 2015-01-01 to 2015-04-01...
📅 Fetching news from 2015-04-02 to 2015-07-01...
📅 Fetching news from 2015-07-02 to 2015-09-30...
📅 Fetching news from 2015-10-01 to 2015-12-30...
📅 Fetching news from 2015-12-31 to 2016-03-30...
📅 Fetching news from 2016-03-31 to 2016-06-29...
📅 Fetching news from 2016-06-30 to 2016-09-28...
📅 Fetching news from 2016-09-29 to 2016-12-28...
📅 Fetching news from 2016-12-29 to 2017-03-29...
📅 Fetching news from 2017-03-30 to 2017-06-28...
📅 Fetching news from 2017-06-29 to 2017-09-27...
📅 Fetching news from 2017-09-28 to 2017-12-27...
📅 Fetching news from 2017-12-28 to 2018-03-28...
📅 Fetching news from 2018-03-29 to 2018-06-27...
📅 Fetching news from 2018-06-28 to 2018-09-26...
📅 Fetching news from 2018-09-27 to 2018-12-26...
📅 Fetching news from 2018-12-27 to 2019-03-27...
📅 Fetching news from 2019-03-28 to 2019-06-26...
📅 Fetching news from 2019-06-27 to 2019-09-25...
📅 Fetching news from 2019-09-26 to 2019-12-25...
📅 Fetching news from

In [16]:
news_df.head()

,category,datetime,headline,id,image,related,source,summary,url
0,company,2024-09-18 21:09:16,Dow Jones Futures: Stock Market Reverses Lower...,129950341,,TSLA,Yahoo,The stock market closed lower after the S&P 50...,https://finnhub.io/api/news?id=7d0c0298b4c8145...
1,company,2024-09-18 20:53:54,How interest rate cuts will impact vehicle sal...,129950342,https://s.yimg.com/ny/api/res/1.2/7OLVHPcVE.xg...,TSLA,Yahoo,"According to a report from Edmunds, 62% of Ame...",https://finnhub.io/api/news?id=c6907c0db94a1a5...
2,company,2024-09-18 20:16:00,These Stocks Moved the Most Today: Intuitive M...,129953101,https://s.yimg.com/ny/api/res/1.2/JCvbirRecNl0...,TSLA,Yahoo,Intuitive Machines is soaring after receiving ...,https://finnhub.io/api/news?id=86f07de230e58f2...
3,company,2024-09-18 20:14:00,Tesla Stock Gains. What History Says Happens o...,129949005,,TSLA,Yahoo,It’s Fed day and Tesla shareholders were wat...,https://finnhub.io/api/news?id=95465e4549dc678...
4,company,2024-09-18 20:07:45,GM electric vehicles now have access to Tesla ...,129950344,https://s.yimg.com/ny/api/res/1.2/SLw7upIftvAk...,TSLA,Yahoo,General Motors (GM) electric vehicles have bee...,https://finnhub.io/api/news?id=fce78c92ae36b71...


In [17]:
news_df.drop(columns=['category','id', 'image', 'related', 'source', 'url'], inplace=True)

In [18]:
news_df['datetime'] = pd.to_datetime(news_df['datetime'])

In [19]:
news_df['Date']= news_df['datetime'].dt.date

In [20]:
news_df.head()

,datetime,headline,summary,Date
0,2024-09-18 21:09:16,Dow Jones Futures: Stock Market Reverses Lower...,The stock market closed lower after the S&P 50...,2024-09-18
1,2024-09-18 20:53:54,How interest rate cuts will impact vehicle sal...,"According to a report from Edmunds, 62% of Ame...",2024-09-18
2,2024-09-18 20:16:00,These Stocks Moved the Most Today: Intuitive M...,Intuitive Machines is soaring after receiving ...,2024-09-18
3,2024-09-18 20:14:00,Tesla Stock Gains. What History Says Happens o...,It’s Fed day and Tesla shareholders were wat...,2024-09-18
4,2024-09-18 20:07:45,GM electric vehicles now have access to Tesla ...,General Motors (GM) electric vehicles have bee...,2024-09-18


In [21]:
news_df['Full_Text'] = news_df['headline'] + ". " + news_df['summary']

In [22]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)  # remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)     # remove punctuation and numbers
    text = re.sub(r'\s+', ' ', text)            # remove extra spaces
    return text.strip()

news_df['Clean_Text'] = news_df['Full_Text'].apply(clean_text)

In [23]:
model_name = "yiyanghkust/finbert-tone"

finbert = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


In [24]:
# Run in batches to avoid memory issues
sentiments = []
batch_size = 16

for i in range(0, len(news_df), batch_size):
    batch_text = news_df['Full_Text'][i:i+batch_size].tolist()
    results = finbert(batch_text)
    sentiments.extend(results)

In [25]:
news_df['sentiment_label'] = [s['label'] for s in sentiments]
news_df['sentiment_score'] = [s['score'] for s in sentiments]


In [26]:
daily_sentiment = news_df.groupby('Date').agg({
    'sentiment_score': 'mean',
    'sentiment_label': lambda x: x.mode()[0],  # most common label
    'headline': 'count'
}).rename(columns={
    'sentiment_score': 'avg_sentiment',
    'headline': 'headline_count'
}).reset_index()


In [27]:
daily_sentiment.head()

,Date,avg_sentiment,sentiment_label,headline_count
0,2024-09-10,0.957516,Neutral,23
1,2024-09-11,0.938024,Neutral,25
2,2024-09-12,0.946918,Neutral,19
3,2024-09-13,0.982804,Neutral,24
4,2024-09-14,0.967986,Neutral,13


In [28]:
newdf = pd.merge(stock_df, daily_sentiment, on='Date', how='left')

In [29]:
newdf['avg_sentiment'] = newdf['avg_sentiment'].fillna(0)
newdf['sentiment_label'] = newdf['sentiment_label'].fillna('NEUTRAL')
newdf['headline_count'] = newdf['headline_count'].fillna(0)


In [30]:
newdf.head()

,Date,Close,High,Low,Open,Volume,avg_sentiment,sentiment_label,headline_count
0,2015-01-02,14.620667,14.883333,14.217333,14.858000,71466000,0.0,NEUTRAL,0.0
1,2015-01-05,14.006000,14.433333,13.810667,14.303333,80527500,0.0,NEUTRAL,0.0
2,2015-01-06,14.085333,14.280000,13.614000,14.004000,93928500,0.0,NEUTRAL,0.0
3,2015-01-07,14.063333,14.318667,13.985333,14.223333,44526000,0.0,NEUTRAL,0.0
4,2015-01-08,14.041333,14.253333,14.000667,14.187333,51637500,0.0,NEUTRAL,0.0


In [31]:
# RSI - using 14-day window (commonly used)
newdf['rsi'] = ta.momentum.RSIIndicator(close=newdf['Close'], window=14).rsi()

# MACD
macd = ta.trend.MACD(close=newdf['Close'])

newdf['macd'] = macd.macd()
newdf['macd_signal'] = macd.macd_signal()
newdf['macd_diff'] = macd.macd_diff()  # Histogram (MACD - Signal)

In [32]:
# Add lagged features
for lag in range(1, 6):
    newdf[f'Close_lag{lag}'] = newdf['Close'].shift(lag)


newdf['prev_close'] = newdf['Close'].shift(1)
newdf['prev_sentiment'] = newdf['avg_sentiment'].shift(1)
newdf['prev_volume'] = newdf['Volume'].shift(1)

In [33]:
# Add more lagged sentiment features
for lag in range(2, 6):
    newdf[f'prev_sentiment_lag{lag}'] = newdf['avg_sentiment'].shift(lag)

In [34]:
# Price change % between days
newdf['daily_return'] = newdf['Close'].pct_change()

# Rolling volatility (e.g., past 5 days)
newdf['volatility_5d'] = newdf['daily_return'].rolling(window=5).std()
newdf['volatility'] = newdf['High'] - newdf['Low']


# Momentum
newdf['momentum_3d'] = newdf['Close'] - newdf['Close'].shift(3)

In [35]:
newdf['weekday'] = pd.to_datetime(newdf['Date']).dt.weekday  # 0=Monday, 6=Sunday

In [36]:
#moving average
newdf['SMA_5'] = newdf['Close'].rolling(5).mean()
newdf['SMA_10'] = newdf['Close'].rolling(10).mean()
newdf['EMA_5'] = newdf['Close'].ewm(span=5).mean()


In [37]:
newdf['High_Low_Ratio'] = newdf['High'] / newdf['Low']
newdf['High_Low_Open_Ratio'] = (newdf['High'] - newdf['Low']) / newdf['Open']
newdf['Close_Open_Ratio'] = newdf['Close'] / newdf['Open']
newdf['Volume_SMA_5'] = newdf['Volume'].rolling(5).mean()
newdf['OBV'] = (newdf['Volume'] * ((newdf['Close'] - newdf['Close'].shift(1)) / abs(newdf['Close'] - newdf['Close'].shift(1)))).fillna(0).cumsum()
newdf['Close_SMA_5_Ratio'] = newdf['Close'] / newdf['SMA_5']
newdf['Close_SMA_10_Ratio'] = newdf['Close'] / newdf['SMA_10']

In [38]:
newdf.shape

(2651, 40)

In [39]:
final_df = newdf.dropna().reset_index(drop=True)
final_df.head()

,Date,Close,High,Low,Open,Volume,avg_sentiment,sentiment_label,headline_count,rsi,...,SMA_5,SMA_10,EMA_5,High_Low_Ratio,High_Low_Open_Ratio,Close_Open_Ratio,Volume_SMA_5,OBV,Close_SMA_5_Ratio,Close_SMA_10_Ratio
0,2015-02-20,14.474000,14.506667,13.987333,14.052000,89731500,0.0,NEUTRAL,0.0,58.709798,...,13.885333,14.054734,14.074018,1.037129,0.036958,1.030031,72061200.0,-261112500.0,1.042395,1.029831
1,2015-02-23,13.822667,14.546667,13.755333,14.377333,127497000,0.0,NEUTRAL,0.0,47.638976,...,13.932933,13.987934,13.990235,1.057529,0.055040,0.961421,78987600.0,-388609500.0,0.992086,0.988185
2,2015-02-24,13.607333,13.819333,13.446667,13.819333,99054000,0.0,NEUTRAL,0.0,44.641846,...,13.929733,13.898800,13.862601,1.027714,0.026967,0.984659,86859600.0,-487663500.0,0.976855,0.979029
3,2015-02-25,13.584000,13.809333,13.505333,13.662667,58642500,0.0,NEUTRAL,0.0,44.316488,...,13.920400,13.815267,13.769734,1.022510,0.022250,0.994242,90447300.0,-546306000.0,0.975834,0.983260
4,2015-02-26,13.812667,14.072667,13.481333,13.600000,97093500,0.0,NEUTRAL,0.0,48.293631,...,13.860133,13.777867,13.784045,1.043863,0.043480,1.015637,94403700.0,-449212500.0,0.996575,1.002526


In [40]:
# final_df['target'] = (final_df['Close'].shift(-1) > final_df['Close']).astype(int)
final_df['return'] = final_df['Close'].pct_change().shift(-1)
final_df['target'] = (final_df['return'] > 0.01).astype(int)


In [41]:
final_df = final_df.dropna().reset_index(drop=True)

In [42]:
final_df.columns

Index(['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'avg_sentiment',
       'sentiment_label', 'headline_count', 'rsi', 'macd', 'macd_signal',
       'macd_diff', 'Close_lag1', 'Close_lag2', 'Close_lag3', 'Close_lag4',
       'Close_lag5', 'prev_close', 'prev_sentiment', 'prev_volume',
       'prev_sentiment_lag2', 'prev_sentiment_lag3', 'prev_sentiment_lag4',
       'prev_sentiment_lag5', 'daily_return', 'volatility_5d', 'volatility',
       'momentum_3d', 'weekday', 'SMA_5', 'SMA_10', 'EMA_5', 'High_Low_Ratio',
       'High_Low_Open_Ratio', 'Close_Open_Ratio', 'Volume_SMA_5', 'OBV',
       'Close_SMA_5_Ratio', 'Close_SMA_10_Ratio', 'return', 'target'],
      dtype='object')

In [43]:
numerical_features = [ 'Close', 'High', 'Low', 'Open', 'Volume', 'avg_sentiment',
       'headline_count', 'rsi', 'macd', 'macd_signal',
       'macd_diff', 'Close_lag1', 'Close_lag2', 'Close_lag3', 'Close_lag4',
       'Close_lag5', 'prev_close', 'prev_sentiment', 'prev_volume',
       'prev_sentiment_lag2', 'prev_sentiment_lag3', 'prev_sentiment_lag4',
       'prev_sentiment_lag5', 'daily_return', 'volatility_5d', 'volatility',
       'momentum_3d', 'weekday', 'SMA_5', 'SMA_10', 'EMA_5', 'High_Low_Ratio',
       'High_Low_Open_Ratio', 'Close_Open_Ratio', 'Volume_SMA_5', 'OBV',
       'Close_SMA_5_Ratio', 'Close_SMA_10_Ratio']

categorical_features = ['sentiment_label']

df_model = final_df[numerical_features + categorical_features + ['target']].dropna()

X = df_model[numerical_features + categorical_features]
y = df_model['target']

In [44]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

In [45]:
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42
    ))
])


In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

model.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [05:06:28] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Close', 'High', 'Low',
                                                   'Open', 'Volume',
                                                   'avg_sentiment',
                                                   'headline_count', 'rsi',
                                                   'macd', 'macd_signal',
                                                   'macd_diff', 'Close_lag1',
                                                   'Close_lag2', 'Close_lag3',
                                                   'Close_lag4', 'Close_lag5',
                                                   'prev_close',
                                                   'prev_sentiment',
                                                   'prev_volume',
                                                   'prev_sentiment_lag2',
                                                   'prev_sentiment_lag3'...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.1,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=100, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [47]:
y_pred = model.predict(X_test)

In [48]:
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.65      0.90      0.76       333\n           1       0.49      0.16      0.24       191\n\n    accuracy                           0.63       524\n   macro avg       0.57      0.53      0.50       524\nweighted avg       0.59      0.63      0.57       524\n'

In [49]:

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc:.4f}")


Accuracy: 0.6336


In [50]:
# import joblib
# joblib.dump(model, 'xgboost_model.pkl')

In [51]:
#final_df.to_csv('final_df.csv', index=False)


In [54]:
import cohere
import pandas as pd

# Authenticate with Cohere
co = cohere.Client("gKOrxLTQztkU7rLIUra670tfDHOUVdSpdt5DJ8w9")  # Replace with your actual key



#select the last row for prediction
row = final_df.iloc[-1]

#Compose a detailed insight prompt
prompt = f"""
Generate a detailed financial insight based on the following data:

- Date: {row['Date']}
- Closing Price: {row['Close']:.2f}
- Sentiment: {row['sentiment_label']} with average score {row['avg_sentiment']:.2f} from {int(row['headline_count'])} news headlines
- RSI: {row['rsi']:.2f}
- MACD: {row['macd']:.2f}
- Model Prediction: {'📈 Rise' if row['target'] == 1 else '📉 Fall'}

Include market interpretation, technical outlook, and investor recommendation in a professional tone.
"""

# Call Cohere's generate endpoint
response = co.generate(
    model="command-r-plus",
    prompt=prompt,
    max_tokens=300,
    temperature=0.7,
)

print(f"\n📘 Insight for {row['Date']}:\n{response.generations[0].text.strip()}")



📘 Insight for 2025-07-17:
### Market Interpretation:

On July 17, 2025, the market closed with a neutral sentiment, as indicated by the average score of 0.93 from 28 news headlines. The closing price of 319.41 reflects a relatively stable position with a slight upward bias. Overall, the market sentiment appears balanced, with neither a strong bullish nor bearish inclination. 

### Technical Outlook:

The Relative Strength Index (RSI) reading of 51.61 suggests that the asset is currently in a healthy range, neither overbought nor oversold. This indicates a potential for continued sideways movement or a mild price correction before any significant rise or decline. 

However, the Moving Average Convergence Divergence (MACD) indicator, with a value of -1.75, suggests a potential downward trend. A negative MACD value indicates that the shorter-term exponential moving average is below the longer-term average, which is typically interpreted as a bearish signal. 

### Model Prediction: 

Desp

In [ ]:
%pip install finnhub-python

In [ ]:
%pip install ta

In [53]:
%pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.9/291.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 44.5 MB/s eta 0:00:00
